# 1: Introduction To The Data
Instructions Import the Pandas library and read auto.csv into a Dataframe named cars.
Use the Series.unique() method to assign the unique elements in the column origin to unique_regions. Then use the print function to display unique_regions.
Hint
DataFrame['a'].unique() will returns the unique values in column a.

In [1]:
import pandas as pd
cars = pd.read_csv("auto1.csv")
cars.head(20)

,mpg,cyls,displacement,hp,weight,acc,year,origin
0,18,8,307,130,3504,12.0,70,1
1,15,8,350,165,3693,11.5,70,1
2,18,8,318,150,3436,11.0,70,1
3,16,8,304,150,3433,12.0,70,1
4,17,8,302,140,3449,10.5,70,1
5,15,8,429,198,4341,10.0,70,1
6,14,8,454,220,4354,9.0,70,1
7,14,8,440,215,4312,8.5,70,1
8,14,8,455,225,4425,10.0,70,1
9,15,8,390,190,3850,8.5,70,1


In [2]:
unique_regions = cars["origin"].unique()
print(unique_regions)

[1 3 2]


# 2: Dummy Variables
Instructions Use the pandas.get_dummies() function to create dummy values from the year column.
Use the prefix attribute to prepend year to each of the resulting column names.
Assign the resulting Dataframe to dummy_years.
Use the pandas.concat() function to concatenate the columns from dummy_years to cars.
Use the DataFrame.drop() method to drop the year and cylinders columns from cars.
Display the first 5 rows of the new cars Dataframe to confirm.
Hint
Make sure to set the axis attribute to 1 when dropping columsn: cars = cars.drop("year", axis=1).

In [3]:
dummy_cylinders = pd.get_dummies(cars["cyls"], prefix="cycl")
cars = pd.concat([cars, dummy_cylinders], axis=1)

dummy_years = pd.get_dummies(cars["year"], prefix="year")
cars = pd.concat([cars, dummy_years], axis=1)

cars = cars.drop("cyls", axis=1)
cars = cars.drop("year", axis=1)

print(cars.head())

   mpg  displacement   hp  weight   acc  origin  cycl_3  cycl_4  cycl_5  \
0   18           307  130    3504  12.0       1       0       0       0   
1   15           350  165    3693  11.5       1       0       0       0   
2   18           318  150    3436  11.0       1       0       0       0   
3   16           304  150    3433  12.0       1       0       0       0   
4   17           302  140    3449  10.5       1       0       0       0   

   cycl_6   ...     year_73  year_74  year_75  year_76  year_77  year_78  \
0       0   ...           0        0        0        0        0        0   
1       0   ...           0        0        0        0        0        0   
2       0   ...           0        0        0        0        0        0   
3       0   ...           0        0        0        0        0        0   
4       0   ...           0        0        0        0        0        0   

   year_79  year_80  year_81  year_82  
0        0        0        0        0  
1        0  

# 3: Multiclass Classification
Instructions Split the shuffled_cars Dataframe into 2 Dataframes: train and test.
Assign the first 70% of the shuffled_cars to train.
Assign the last 30% of the shuffled_cars to test.
Hint
Multiply the number of rows in the cars Dataframe by 0.7 and use that value to segment the data. Read about loc vs iloc here.

In [4]:
import numpy as np
shuffled_rows = np.random.permutation(cars.index)
shuffled_cars = cars.iloc[shuffled_rows]

highest_train_row = int(cars.shape[0] * .70)
train = shuffled_cars.iloc[0:highest_train_row]
test= shuffled_cars.iloc[highest_train_row:]

print(train)
print(test)

      mpg  displacement   hp  weight   acc  origin  cycl_3  cycl_4  cycl_5  \
314  19.1           225   90    3381  18.7       1       0       0       0   
355  30.7           145   76    3160  19.6       2       0       0       0   
385  36.0           135   84    2370  13.0       1       0       1       0   
257  20.8           200   85    3070  16.7       1       0       0       0   
317  31.3           120   75    2542  17.5       3       0       1       0   
338  30.0           135   84    2385  12.9       1       0       1       0   
59   20.0           140   90    2408  19.5       1       0       1       0   
327  44.6            91   67    1850  13.8       3       0       1       0   
280  22.3           140   88    2890  17.3       1       0       1       0   
203  28.0            97   75    2155  16.4       3       0       1       0   
250  19.2           231  105    3535  19.2       1       0       0       0   
252  20.2           200   85    2965  15.8       1       0      

# 4. Training A Multiclass Logistic Regression Model
Instructions For each value in unique_origins, train a logistic regression model with the following parameters:
X: Dataframe containing just the cylinder & year binary columns.
y: list (or Series) of Boolean values:
True if observation's value for origin matches the current iterator variable.
False if observation's value for origin doesn't match the current iterator variable.
Add each model to the models dictionary with the following structure:
key: origin value (1, 2, or 3),
value: relevant LogistcRegression model instance.
Hint
Use thestr.startswith method to check if a string starts with a substring.
Documentation for LogisticRegression.fit(

In [19]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()

unique_origins = cars["origin"].unique()
unique_origins.sort()

models = {}

features = [c for c in train.columns if c.startswith("cyl") or c.startswith("year")]

for origin in unique_origins:
    model = LogisticRegression()
    
    X_train = train[features]
    Y_train = train["origin"] == origin
    
    model.fit(X_train, Y_train)
    models[origin] = model

# 5. Testing The Models
Instructions For each origin value from unique_origins:
Use the LogisticRegression predict_proba function to return the 3 lists of predicted probabilities for the test set and add to the testing_probs Dataframe.
Hint
testing_probs[1] should return the probability returned from model 1 for each observation.

In [27]:
testing_probs = pd.DataFrame(columns=unique_origins)

for origin in unique_origins:
    X_test = test[features]
    
    testing_probs[origin] = models[origin].predict_proba(X_test)[:,1]
print(testing_probs)

            1         2         3
0    0.735029  0.218407  0.073244
1    0.735029  0.218407  0.073244
2    0.659705  0.186295  0.160186
3    0.620247  0.150779  0.233208
4    0.274407  0.271299  0.446166
5    0.648055  0.081102  0.287575
6    0.672548  0.179801  0.154763
7    0.274407  0.271299  0.446166
8    0.659705  0.186295  0.160186
9    0.648055  0.081102  0.287575
10   0.659705  0.186295  0.160186
11   0.659705  0.117067  0.233208
12   0.649892  0.188398  0.166161
13   0.620247  0.150779  0.233208
14   0.649892  0.188398  0.166161
15   0.735029  0.218407  0.073244
16   0.620247  0.150779  0.233208
17   0.544143  0.125313  0.336180
18   0.596571  0.179801  0.224778
19   0.595761  0.175563  0.230241
20   0.797203  0.091336  0.134981
21   0.797203  0.091336  0.134981
22   0.648055  0.081102  0.287575
23   0.544143  0.125313  0.336180
24   0.649892  0.188398  0.166161
25   0.648055  0.081102  0.287575
26   0.648055  0.081102  0.287575
27   0.747355  0.158093  0.108284
28   0.797203 

# 6. Choose The Origin
Instructions Classify each observation in the test set using the testing_probs Dataframe.
Assign the predicted origins to predicted_origins and use the print function to display it.
Hint
Use the .idxmax(axis=1) method to find the classifications.

In [29]:
predicted_origins = testing_probs.idxmax(axis=1)
print(predicted_origins)

0      1
1      1
2      1
3      1
4      3
5      1
6      1
7      3
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
88     1
89     1
90     1
91     1
92     1
93     1
94     1
95     1
96     1
97     3
98     1
99     1
100    1
101    3
102    1
103    1
104    1
105    1
106    1
107    1
108    1
109    1
110    1
111    1
112    1
113    1
114    1
115    1
116    1
117    1
dtype: int64
